# Clustering Food Venues in Canadian Cities

This notebook will explore postal codes and food venues in Canada's three largest cities: Toronto, Montreal and Calgary.

Our objective is to visualize where different categories of food venues are clustered, which may have some implications for the surrounding population demographics and business environment.

(Note: Forward Sortation Areas (FSA) are the first three letters of Canadian Postal Codes. But here we use the term interchangeably with Postal Code or Post Code.)

#### Import everything we need.

In [1]:
# dill for saving jupyter session offline
try: 
    import dill
except: 
    !conda install -c conda-forge dill --yes # install only if necessary
    import dill
# dill.dump_session('notebook_env.db') # save notebook kernel session
# dill.load_session('notebook_env.db') # load notebook kernel session

In [4]:
# import libraries

# time for performance measures
import time

# numpy for data vectors
import numpy as np

# pandas for data analysis
import pandas as pd
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

from collections import Counter

# # transform json into pandas dataframe
# from pandas.io.json import json_normalize

# json
import json

# requests
import requests

# nominatim to convert an address into latitude and longitude values
try:
    from geopy.geocoders import Nominatim 
except (ImportError, ModuleNotFoundError): #install only if necessary
    !conda install -c conda-forge geopy --yes 
    from geopy.geocoders import Nominatim

# Matplotlib for plotting
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as py

# import k-means from sklearn clustering
from sklearn.cluster import KMeans

# folium for visualizaing maps
try:
    import folium
except (ImportError, ModuleNotFoundError): #install only if necessary
    !conda install -c conda-forge folium=0.5.0 --yes
    import folium # plotting library
from folium.plugins import MarkerCluster    
from folium.plugins import FastMarkerCluster

## Postal Code Data

I am using wikipedia tables of postal codes in Canada:  
1. Toronto: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M  
1. Montreal: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_H  
1. Calgary: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_T  

In [135]:
# first dataframe of cities and letter codes
city_codes = pd.DataFrame([['M','Toronto, Ontario'],['H','Montreal, Quebec'],['T', 'Calgary, Alberta']],
                         columns = ['Letter', 'City'])
city_codes

,Letter,City
0,M,"Toronto, Ontario"
1,H,"Montreal, Quebec"
2,T,"Calgary, Alberta"


In [136]:
# scrape html tables
fsa_set=[]

for url in ('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_' + letter for letter in city_codes.iloc[:,0]):

    html = requests.get(url).content
    assert html[:15].decode("utf-8").lower()=='<!doctype html>', 'HTML required' # Check for html format, if not raise error
    df = pd.read_html(html)[0] #read html table as df 
    
    # 'M' postal code table is usable as is, other tables need to be reshaped.
    
    if 'Postcode' not in df.columns: # grid without headers
        pc_list = []
        for c in df.columns:
            pc_list.extend(list(df[c])) # stack columns
            
        fsa = [r[:3] for r in pc_list] # split strings (first 3 letters)
        descr = [r[3:] for r in pc_list] # split strings (the rest)
        df = pd.DataFrame(zip(fsa,descr)) # recreate dataframe

        if url[-1] == 'T': # T's include all Alberta. Exclude all but Calgary
            df = df[df.iloc[:,1].str.contains('Calgary', case=False)]
            
    df = df[df.iloc[:,1].str.contains('not assigned', case=False)==False] # Exclude not assigned
    
    fsa_set.extend(df.iloc[:,0]) # add to list of fsa

fsa_set = sorted(set(fsa_set)) # sorted list of unique fsa

print('{} forward sortation areas found.'.format(len(fsa_set)))
fsa_set[0:5]

261 forward sortation areas found.


['H0H', 'H0M', 'H1A', 'H1B', 'H1C']

Although we excluded postal codes designated as 'not assigned', we still have some that are not in use. 

We will be using Foursquare's API to get venues near the postal codes. Any that are not found by FourSquare's geocoder will be ignored at that time.

## Foursquare Data

#### Define Foursquare Credentials and Version

*(credentials removed in public copy)*

In [137]:
# CLIENT_ID = 'XXX' # Foursquare ID
# CLIENT_SECRET = 'XXX' # Foursquare Secret
# VERSION = '20180605' # Foursquare API version

In [138]:
CLIENT_ID = 'TTJ4LSILREWCDMCJOCXMTGHVJBYIFD0H5K10WVPYIBUOMWQ5' # Foursquare ID
CLIENT_SECRET = '14RPHQIVUD4MQZBBVZ0LX3K0ZF1OVXLVNVJREKMCFCMO0QYT' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Use a function to call for the Foursquare API with a search of the closest venues within a radius of each postal code.

Here I will focus only on the food category venues, which is one of the most common and diverse categories on Foursquare. For simplicity I will only look at the primary category of each venue, as some venues have multiple categorizations.

The Foursquare Search endpoint returns the nearest venues to the point of origin, matching the query. I can also add a maximum radius in meters and a maximum number of returned venues.

In [139]:
# function that gets foursquare SEARCH results
def search_venues_fx(places, radius=500, limit=50, categoryId='4d4b7105d754a06374d81259'): 
# max radius=100,000; max limit=50; categoryID: food='4d4b7105d754a06374d81259'
    
    venues_list=[]
    i=0 # counter
    for place in places:
        print(i, place)
        # create the API request URL
        url = ('https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}'
               '&near={}&radius={}&limit={}&categoryId={}').format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                place, 
                radius, 
                limit,
                categoryId)

        # make the GET request
        try: 
            response = requests.get(url).json()['response']
            response['venues'] # check for venues
            coords = response['geocode']['feature']['geometry']['center'] # return geo coords
        except:
            continue # if no results, skip to the next
               
        # return relevant information for each venue and add to list
        venues_list.append([(
            place[0:3], #FSA only
            coords['lat'],
            coords['lng'], 
            v['id'], 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'], 
            v['categories'][0]['name'],
            v['categories'][0]['id'] # very few venues have multiple categories, get only the first
            ) for v in response['venues']])
        
        i+=1 # count successful queries

    # create dataframe from venue list
    df = pd.DataFrame(venue for place_venues in venues_list for venue in place_venues)
    df.columns = [
        'Place', 
        'Place Latitude', 
        'Place Longitude', 
        'Id',
        'Name', 
        'Latitude', 
        'Longitude', 
        'Category',
        'Category Id'
        ]
    
    print('{} places queried. {} venues found.'.format(i, len(df)))
    return(df)

In [140]:
# sample of FSAs for testing
import random
# fsa_sample = random.sample([x for x in fsa_set if x.startswith('M')],k=10)
fsa_sample = random.sample(fsa_set,k=30)
fsa_sample

['M4V',
 'H2E',
 'H7B',
 'H3B',
 'H9C',
 'H4W',
 'M1K',
 'H8T',
 'M9V',
 'H2J',
 'T3K',
 'H4P',
 'M4Y',
 'H1C',
 'H4V',
 'T1Y',
 'M5A',
 'M1W',
 'H9X',
 'H1X',
 'M9N',
 'T2N',
 'H2S',
 'M6P',
 'H1S',
 'H4A',
 'M1N',
 'T2S',
 'H7W',
 'M5W']

Now I run the function searching our postal codes in the format 'M1A, Canada'. This is sufficient for Foursquare to geocode the postal code and return the venues, as well as the coordinates for our later use.

I've set the radius to 1km, which looks like more than enough to cover for the area of most postal codes in Toronto. I've also set the limit to 50 results each, which is the documented maximum. I think this should provide a reasonable representation of the majority of food venues throughout the city.

In [141]:
t1 = time.perf_counter()

results = search_venues_fx( places=(pc+', Canada' for pc in fsa_set), radius=1000, limit=50)

t2 = time.perf_counter()
print('{:0.2f}s elapsed.'.format(t2-t1))

0 H0H, Canada
1 H0M, Canada
2 H1A, Canada
3 H1B, Canada
4 H1C, Canada
5 H1E, Canada
6 H1G, Canada
7 H1H, Canada
8 H1J, Canada
9 H1K, Canada
10 H1L, Canada
11 H1M, Canada
12 H1N, Canada
13 H1P, Canada
14 H1R, Canada
15 H1S, Canada
16 H1T, Canada
17 H1V, Canada
18 H1W, Canada
19 H1X, Canada
20 H1Y, Canada
21 H1Z, Canada
22 H2A, Canada
23 H2B, Canada
24 H2C, Canada
25 H2E, Canada
26 H2G, Canada
27 H2H, Canada
28 H2J, Canada
29 H2K, Canada
30 H2L, Canada
31 H2M, Canada
32 H2N, Canada
33 H2P, Canada
34 H2R, Canada
35 H2S, Canada
36 H2T, Canada
37 H2V, Canada
38 H2W, Canada
39 H2X, Canada
40 H2Y, Canada
41 H2Z, Canada
42 H3A, Canada
43 H3B, Canada
44 H3C, Canada
45 H3E, Canada
46 H3G, Canada
47 H3H, Canada
48 H3J, Canada
49 H3K, Canada
50 H3L, Canada
51 H3M, Canada
52 H3N, Canada
53 H3P, Canada
54 H3R, Canada
55 H3S, Canada
56 H3T, Canada
57 H3V, Canada
58 H3W, Canada
59 H3X, Canada
60 H3Y, Canada
61 H3Z, Canada
62 H4A, Canada
63 H4B, Canada
64 H4C, Canada
65 H4E, Canada
66 H4G, Canada
67 H4

In [148]:
# dill for saving jupyter session offline
try: 
    import dill
except: 
    !conda install -c conda-forge dill --yes # install only if necessary
    import dill
# dill.dump_session('notebook_env.db') # save notebook kernel session
# dill.load_session('notebook_env.db') # load notebook kernel session

#### Check the resulting dataframes.

In [246]:
print(results.shape)
results.head()

(9080, 10)


,Place,Place Latitude,Place Longitude,Id,Name,Latitude,Longitude,Category,Category Id,Letter
0,H0M,45.6986,-73.5025,4c12f4a3a5eb76b0d93abfb7,Tenuta,45.694548,-73.509593,Italian Restaurant,4bf58dd8d48988d110941735,H
1,H0M,45.6986,-73.5025,59149185237dee771d29ecfb,Dairy Queen Store,45.696467,-73.492754,Fast Food Restaurant,4bf58dd8d48988d16e941735,H
2,H0M,45.6986,-73.5025,4eee543d93add02fcd2d9b43,Tim Hortons,45.690808,-73.496677,Coffee Shop,4bf58dd8d48988d1e0931735,H
3,H0M,45.6986,-73.5025,4e9936fde5fa393c4e31bf42,McDonald's,45.691514,-73.494216,Fast Food Restaurant,4bf58dd8d48988d16e941735,H
4,H0M,45.6986,-73.5025,4bf884a25efe2d7fb3f86a34,Dairy Queen,45.696294,-73.491945,Fast Food Restaurant,4bf58dd8d48988d16e941735,H


#### Add the cities.

In [247]:
# merge venues to cities using the first letter of postal code
place_venues = results
place_venues['Letter'] = [p[0] for p in place_venues['Place']] # create a column for the first letter of the fsa
place_venues = pd.merge(city_codes[['Letter', 'City']], place_venues, left_on='Letter', right_on='Letter') # merge on the letter
place_venues.drop('Letter', axis=1, inplace=True) # drop the letter column

print(place_venues.shape)
place_venues.head()

(9080, 10)


,City,Place,Place Latitude,Place Longitude,Id,Name,Latitude,Longitude,Category,Category Id
0,"Toronto, Ontario",M1B,43.8113,-79.193,517dcdb6f1363b7a770a8424,Meena's Fine Foods,43.804476,-79.199753,Indian Restaurant,4bf58dd8d48988d10f941735
1,"Toronto, Ontario",M1B,43.8113,-79.193,5914a5d16a8d866b54e07aed,Dairy Queen (Treat),43.818728,-79.185368,Ice Cream Shop,4bf58dd8d48988d1c9941735
2,"Toronto, Ontario",M1B,43.8113,-79.193,4bb6b9446edc76b0d771311c,Wendy's,43.807448,-79.199056,Fast Food Restaurant,4bf58dd8d48988d16e941735
3,"Toronto, Ontario",M1B,43.8113,-79.193,4f5d15b7e4b06784f91bad71,Second Cup,43.802165,-79.196114,Coffee Shop,4bf58dd8d48988d1e0931735
4,"Toronto, Ontario",M1B,43.8113,-79.193,5d013e146a5950002cad73ea,Pizza Pizza,43.819633,-79.184681,Pizza Place,4bf58dd8d48988d1ca941735


#### Separate just the postal code coordinates.

In [248]:
# create a table of just postcode coordinates
places_coords = place_venues.iloc[:,:4].drop_duplicates()
places_coords.reset_index(drop=True, inplace=True)

print(places_coords.shape)
places_coords.head()

(242, 4)


,City,Place,Place Latitude,Place Longitude
0,"Toronto, Ontario",M1B,43.8113,-79.1930
1,"Toronto, Ontario",M1C,43.7878,-79.1564
2,"Toronto, Ontario",M1E,43.7678,-79.1866
3,"Toronto, Ontario",M1G,43.7712,-79.2144
4,"Toronto, Ontario",M1H,43.7686,-79.2389


#### Explore the venues data.

Let's check how many venues were returned for each postal code.

In [249]:
place_venues[['City', 'Place','Name']].groupby(['City','Place']).count().head(10)

Name
City             Place      
Calgary, Alberta T1Y      31
                 T2A      28
                 T2B       9
                 T2C      21
                 T2E      26
                 T2G      38
                 T2H      50
                 T2K      50
                 T2N      35
                 T2R      50

Let's find out how many unique food categories make up our venues.

In [250]:
print('There are {} unique categories across {} venues.'.format(
    len(place_venues['Category'].unique()), 
    place_venues.shape[0]))

There are 195 unique categories across 9080 venues.


In [251]:
place_venues['Category'].value_counts().head(10)

Coffee Shop             1104
Café                     654
Fast Food Restaurant     554
Pizza Place              530
Restaurant               463
Bakery                   362
Italian Restaurant       280
Breakfast Spot           241
Sandwich Place           238
Chinese Restaurant       214
Name: Category, dtype: int64

We can see that Coffee Shops and Cafes dominate the data, taking up nearly a fifth of all venues.

In [252]:
# remove coffee shop and cafe
place_venues = place_venues[(place_venues['Category']!='Coffee Shop') & (place_venues['Category']!='Café')]

print(place_venues.shape)
place_venues['Category'].value_counts().head(10)

(7322, 10)


Fast Food Restaurant    554
Pizza Place             530
Restaurant              463
Bakery                  362
Italian Restaurant      280
Breakfast Spot          241
Sandwich Place          238
Chinese Restaurant      214
Asian Restaurant        179
Sushi Restaurant        172
Name: Category, dtype: int64

#### Add in the higher level categories from Foursquare.

In [253]:
# get categories json
url= ('https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'
      .format( CLIENT_ID, CLIENT_SECRET, VERSION))
response = requests.get(url).json()['response']

In [254]:
# create a dataframe of categories from the json

# loop through each level of hierarchy, drilling down to get all subcategories
categories_list = []
for c1 in response['categories']:
    if c1['name'].lower()!='food': # food category only
        continue
        
    else:
        # if no subcategories, copy category across all subsequent subcategories
        categories_list.append((
            c1['id'], c1['name'],
            c1['id'], c1['name'],
            c1['id'], c1['name'],
            c1['id'], c1['name'],
            c1['id'], c1['name']))
            
        if len( c1['categories']) > 0: # if subcategories exist, go to next level
            for c2 in c1['categories']:
                categories_list.append((
                    c1['id'], c1['name'],
                    c2['id'], c2['name'],
                    c2['id'], c2['name'],
                    c2['id'], c2['name'],
                    c2['id'], c2['name']))
                    
                if len( c2['categories']) > 0:                    
                    for c3 in c2['categories']:
                        categories_list.append((
                            c1['id'], c1['name'],
                            c2['id'], c2['name'],
                            c3['id'], c3['name'],
                            c3['id'], c3['name'],
                            c3['id'], c3['name']))
                            
                        if len( c3['categories']) > 0:
                            for c4 in c3['categories']:
                                categories_list.append((
                                    c1['id'], c1['name'],
                                    c2['id'], c2['name'],
                                    c3['id'], c3['name'],
                                    c4['id'], c4['name'],
                                    c4['id'], c4['name']))
                                    
                                if len( c4['categories']) > 0:
                                    for c5 in c4['categories']:
                                        categories_list.append((
                                            c1['id'], c1['name'],
                                            c2['id'], c2['name'],
                                            c3['id'], c3['name'],
                                            c4['id'], c4['name'],
                                            c5['id'], c5['name']))

# create dataframe of hierarchy levels as numbered columns, one row per category or subcategory
categories_wide = pd.DataFrame(categories_list)
categories_wide.columns = [
    'id1',
    'name1',
    'id2',
    'name2',
    'id3',
    'name3',
    'id4',
    'name4',
    'id5',
    'name5']

print(categories_wide.shape)
categories_wide.head()

(348, 10)


,id1,name1,id2,name2,id3,name3,id4,name4,id5,name5
0,4d4b7105d754a06374d81259,Food,4d4b7105d754a06374d81259,Food,4d4b7105d754a06374d81259,Food,4d4b7105d754a06374d81259,Food,4d4b7105d754a06374d81259,Food
1,4d4b7105d754a06374d81259,Food,503288ae91d4c4b30a586d67,Afghan Restaurant,503288ae91d4c4b30a586d67,Afghan Restaurant,503288ae91d4c4b30a586d67,Afghan Restaurant,503288ae91d4c4b30a586d67,Afghan Restaurant
2,4d4b7105d754a06374d81259,Food,4bf58dd8d48988d1c8941735,African Restaurant,4bf58dd8d48988d1c8941735,African Restaurant,4bf58dd8d48988d1c8941735,African Restaurant,4bf58dd8d48988d1c8941735,African Restaurant
3,4d4b7105d754a06374d81259,Food,4bf58dd8d48988d1c8941735,African Restaurant,4bf58dd8d48988d10a941735,Ethiopian Restaurant,4bf58dd8d48988d10a941735,Ethiopian Restaurant,4bf58dd8d48988d10a941735,Ethiopian Restaurant
4,4d4b7105d754a06374d81259,Food,4bf58dd8d48988d14e941735,American Restaurant,4bf58dd8d48988d14e941735,American Restaurant,4bf58dd8d48988d14e941735,American Restaurant,4bf58dd8d48988d14e941735,American Restaurant


In [255]:
# pivot the wide table so that all the category levels become additional rows in the same column
categories = pd.concat([categories_wide[['name2','name3','name4','name5']]], axis=1)
categories = categories.stack()
categories.index = categories.index.droplevel(1) #remove index level created by stack
categories.name = 'categories'
categories = pd.merge(categories_wide['id5'], categories, left_index=True, right_index=True) #merge on index
categories.drop_duplicates(inplace=True)
categories.columns = ['Category Id', 'Categories']

print(categories.shape)
categories.head()

(708, 2)


,Category Id,Categories
0,4d4b7105d754a06374d81259,Food
1,503288ae91d4c4b30a586d67,Afghan Restaurant
2,4bf58dd8d48988d1c8941735,African Restaurant
3,4bf58dd8d48988d10a941735,African Restaurant
3,4bf58dd8d48988d10a941735,Ethiopian Restaurant


#### Join the higher level categories to the venues data.

Create a separate table for just unique venues.

In [256]:
# new dataframe of venues only
venues = place_venues.iloc[:,4:].drop_duplicates()

print('{} duplicate venues dropped. {} venues remain.'.format( len(place_venues)-len(venues), len(venues)))
venues.head()

1243 duplicate venues dropped. 6079 venues remain.


,Id,Name,Latitude,Longitude,Category,Category Id
0,517dcdb6f1363b7a770a8424,Meena's Fine Foods,43.804476,-79.199753,Indian Restaurant,4bf58dd8d48988d10f941735
1,5914a5d16a8d866b54e07aed,Dairy Queen (Treat),43.818728,-79.185368,Ice Cream Shop,4bf58dd8d48988d1c9941735
2,4bb6b9446edc76b0d771311c,Wendy's,43.807448,-79.199056,Fast Food Restaurant,4bf58dd8d48988d16e941735
4,5d013e146a5950002cad73ea,Pizza Pizza,43.819633,-79.184681,Pizza Place,4bf58dd8d48988d1ca941735
5,4c069a410ed3c9285848787d,Twiga Snack Bar,43.819914,-79.186483,African Restaurant,4bf58dd8d48988d1c8941735


After merging we should have more rows now that the extra category entries are added for each venue. We may also lose some rows if any venues our search didn't have a food category as their primary category.

In [257]:
venue_categories = pd.merge(venues, categories, on='Category Id') # merge on category Id

print(venue_categories.shape)
venue_categories.head()

(7207, 7)


,Id,Name,Latitude,Longitude,Category,Category Id,Categories
0,517dcdb6f1363b7a770a8424,Meena's Fine Foods,43.804476,-79.199753,Indian Restaurant,4bf58dd8d48988d10f941735,Indian Restaurant
1,56c27e75cd10dd4574dbe02a,Hakka No.1,43.772340,-79.185480,Indian Restaurant,4bf58dd8d48988d10f941735,Indian Restaurant
2,4ef38984d5fb0c021e0fcac3,Indias Flavor,43.766228,-79.191298,Indian Restaurant,4bf58dd8d48988d10f941735,Indian Restaurant
3,4fc6c295e4b0c5a82f0465f8,Al-Hamd Biryani & Pizza,43.767585,-79.219570,Indian Restaurant,4bf58dd8d48988d10f941735,Indian Restaurant
4,5065d8afe4b0a44a76c13445,Shalimar Sweets & Samosa,43.762931,-79.209521,Indian Restaurant,4bf58dd8d48988d10f941735,Indian Restaurant


## Analyze Categories

#### Use one hot encoding to quantify categorical data, including venues with multiple categories.

Using one hot encoding to count the categories that each venue belongs to. I then group by venue id, so we're back to one row per venue, but the extra categories included using <code>max()</code>.

We should end up with slightly less than our original count of venues due to some mismatching categories.

In [258]:
# one hot encoding categories
venues_onehot = pd.get_dummies(venue_categories[['Categories']])

# add id back to dataframe
venues_onehot = pd.merge(venue_categories['Id'], venues_onehot, left_index=True, right_index=True) #merge on index
venues_onehot = venues_onehot.groupby('Id').max().reset_index()

print(venues_onehot.shape)
venues_onehot.head()

(5798, 140)


,Id,Categories_Afghan Restaurant,Categories_African Restaurant,Categories_American Restaurant,Categories_Arepa Restaurant,Categories_Argentinian Restaurant,Categories_Asian Restaurant,Categories_BBQ Joint,Categories_Bagel Shop,Categories_Bakery,Categories_Belgian Restaurant,Categories_Bistro,Categories_Brazilian Restaurant,Categories_Breakfast Spot,Categories_Bubble Tea Shop,Categories_Buffet,Categories_Burger Joint,Categories_Burmese Restaurant,Categories_Burrito Place,Categories_Cafeteria,Categories_Cajun / Creole Restaurant,Categories_Cambodian Restaurant,Categories_Cantonese Restaurant,Categories_Caribbean Restaurant,Categories_Chinese Breakfast Place,Categories_Chinese Restaurant,Categories_Colombian Restaurant,Categories_Comfort Food Restaurant,Categories_Creperie,Categories_Cuban Restaurant,Categories_Cupcake Shop,Categories_Deli / Bodega,Categories_Dessert Shop,Categories_Dim Sum Restaurant,Categories_Diner,Categories_Doner Restaurant,Categories_Dongbei Restaurant,Categories_Donut Shop,Categories_Dumpling Restaurant,Categories_Eastern European Restaurant,Categories_Egyptian Restaurant,Categories_Empanada Restaurant,Categories_English Restaurant,Categories_Ethiopian Restaurant,Categories_Falafel Restaurant,Categories_Fast Food Restaurant,Categories_Filipino Restaurant,Categories_Fish & Chips Shop,Categories_Fondue Restaurant,Categories_Food,Categories_Food Court,Categories_Food Stand,Categories_Food Truck,Categories_French Restaurant,Categories_Fried Chicken Joint,Categories_Frozen Yogurt Shop,Categories_Gastropub,Categories_German Restaurant,Categories_Gluten-free Restaurant,Categories_Greek Restaurant,Categories_Hakka Restaurant,Categories_Halal Restaurant,Categories_Hawaiian Restaurant,Categories_Hot Dog Joint,Categories_Hotpot Restaurant,Categories_Hungarian Restaurant,Categories_Ice Cream Shop,Categories_Indian Chinese Restaurant,Categories_Indian Restaurant,Categories_Indonesian Restaurant,Categories_Irish Pub,Categories_Israeli Restaurant,Categories_Italian Restaurant,Categories_Japanese Restaurant,Categories_Jewish Restaurant,Categories_Juice Bar,Categories_Kebab Restaurant,Categories_Korean Restaurant,Categories_Kosher Restaurant,Categories_Latin American Restaurant,Categories_Lebanese Restaurant,Categories_Mac & Cheese Joint,Categories_Malay Restaurant,Categories_Mediterranean Restaurant,Categories_Mexican Restaurant,Categories_Middle Eastern Restaurant,Categories_Modern European Restaurant,Categories_Mongolian Restaurant,Categories_Moroccan Restaurant,Categories_New American Restaurant,Categories_Noodle House,Categories_North Indian Restaurant,Categories_Pakistani Restaurant,Categories_Pastry Shop,Categories_Peking Duck Restaurant,Categories_Persian Restaurant,Categories_Peruvian Restaurant,Categories_Pet Café,Categories_Pie Shop,Categories_Pizza Place,Categories_Poke Place,Categories_Polish Restaurant,Categories_Portuguese Restaurant,Categories_Poutine Place,Categories_Ramen Restaurant,Categories_Restaurant,Categories_Russian Restaurant,Categories_Salad Place,Categories_Salvadoran Restaurant,Categories_Sandwich Place,Categories_Scandinavian Restaurant,Categories_Seafood Restaurant,Categories_Shanghai Restaurant,Categories_Shawarma Place,Categories_Snack Place,Categories_Soup Place,Categories_South American Restaurant,Categories_South Indian Restaurant,Categories_Southern / Soul Food Restaurant,Categories_Souvlaki Shop,Categories_Spanish Restaurant,Categories_Sri Lankan Restaurant,Categories_Steakhouse,Categories_Sushi Restaurant,Categories_Swiss Restaurant,Categories_Szechuan Restaurant,Categories_Taco Place,Categories_Taiwanese Restaurant,Categories_Tapas Restaurant,Categories_Tea Room,Categories_Tex-Mex Restaurant,Categories_Thai Restaurant,Categories_Theme Restaurant,Categories_Tibetan Restaurant,Categories_Turkish Restaurant,Categories_Udon Restaurant,Categories_Vegetarian / Vegan Restaurant,Categories_Vietnamese Restaurant,Categories_Wings Joint,Categories_Xinjiang Restaurant
0,4acf2eb1f964a5209fd220e3,

Check the final list of venues.

In [259]:
# merge final venue ids back to dataframe with latitude, longitude
venues_kept = pd.merge(venues_onehot['Id'], place_venues[['Id', 'City', 'Name', 'Latitude', 'Longitude', 'Category']], 
                    left_on='Id', right_on='Id')
venues_kept.drop_duplicates(inplace=True) # drop duplicates
venues_kept.reset_index(drop=True, inplace=True)

print(venues_kept.shape)
venues_kept.head()

(5798, 6)


,Id,City,Name,Latitude,Longitude,Category
0,4acf2eb1f964a5209fd220e3,"Toronto, Ontario",UFO Restaurant,43.643999,-79.408767,Diner
1,4ad4c05cf964a5200ef620e3,"Toronto, Ontario",Cafe Diplomatico,43.655309,-79.413886,Italian Restaurant
2,4ad4c05cf964a5200ff620e3,"Toronto, Ontario",Fresh On Spadina,43.648048,-79.396008,Vegetarian / Vegan Restaurant
3,4ad4c05cf964a520cbf520e3,"Toronto, Ontario",Safari Bar and Grill,43.729051,-79.418109,American Restaurant
4,4ad4c05cf964a520ccf520e3,"Toronto, Ontario",Auberge du Pommier,43.746962,-79.407879,French Restaurant


#### Group by postal code.

Now I repeat the process but group by FSA and calculating the mean. The mean represents the proportion of venues of each category, out of all the venues near the postal code. This will be our metric used for clustering analysis.

In [260]:
# add postcode back to dataframe
places_onehot = pd.merge(place_venues[['Place','Id']], venues_onehot, left_on='Id', right_on='Id') #merge on index

# group and calculate proportion of each category
places_mean = places_onehot.groupby('Place').mean().reset_index()

print(places_mean.shape)
places_mean.head()

(241, 140)


,Place,Categories_Afghan Restaurant,Categories_African Restaurant,Categories_American Restaurant,Categories_Arepa Restaurant,Categories_Argentinian Restaurant,Categories_Asian Restaurant,Categories_BBQ Joint,Categories_Bagel Shop,Categories_Bakery,Categories_Belgian Restaurant,Categories_Bistro,Categories_Brazilian Restaurant,Categories_Breakfast Spot,Categories_Bubble Tea Shop,Categories_Buffet,Categories_Burger Joint,Categories_Burmese Restaurant,Categories_Burrito Place,Categories_Cafeteria,Categories_Cajun / Creole Restaurant,Categories_Cambodian Restaurant,Categories_Cantonese Restaurant,Categories_Caribbean Restaurant,Categories_Chinese Breakfast Place,Categories_Chinese Restaurant,Categories_Colombian Restaurant,Categories_Comfort Food Restaurant,Categories_Creperie,Categories_Cuban Restaurant,Categories_Cupcake Shop,Categories_Deli / Bodega,Categories_Dessert Shop,Categories_Dim Sum Restaurant,Categories_Diner,Categories_Doner Restaurant,Categories_Dongbei Restaurant,Categories_Donut Shop,Categories_Dumpling Restaurant,Categories_Eastern European Restaurant,Categories_Egyptian Restaurant,Categories_Empanada Restaurant,Categories_English Restaurant,Categories_Ethiopian Restaurant,Categories_Falafel Restaurant,Categories_Fast Food Restaurant,Categories_Filipino Restaurant,Categories_Fish & Chips Shop,Categories_Fondue Restaurant,Categories_Food,Categories_Food Court,Categories_Food Stand,Categories_Food Truck,Categories_French Restaurant,Categories_Fried Chicken Joint,Categories_Frozen Yogurt Shop,Categories_Gastropub,Categories_German Restaurant,Categories_Gluten-free Restaurant,Categories_Greek Restaurant,Categories_Hakka Restaurant,Categories_Halal Restaurant,Categories_Hawaiian Restaurant,Categories_Hot Dog Joint,Categories_Hotpot Restaurant,Categories_Hungarian Restaurant,Categories_Ice Cream Shop,Categories_Indian Chinese Restaurant,Categories_Indian Restaurant,Categories_Indonesian Restaurant,Categories_Irish Pub,Categories_Israeli Restaurant,Categories_Italian Restaurant,Categories_Japanese Restaurant,Categories_Jewish Restaurant,Categories_Juice Bar,Categories_Kebab Restaurant,Categories_Korean Restaurant,Categories_Kosher Restaurant,Categories_Latin American Restaurant,Categories_Lebanese Restaurant,Categories_Mac & Cheese Joint,Categories_Malay Restaurant,Categories_Mediterranean Restaurant,Categories_Mexican Restaurant,Categories_Middle Eastern Restaurant,Categories_Modern European Restaurant,Categories_Mongolian Restaurant,Categories_Moroccan Restaurant,Categories_New American Restaurant,Categories_Noodle House,Categories_North Indian Restaurant,Categories_Pakistani Restaurant,Categories_Pastry Shop,Categories_Peking Duck Restaurant,Categories_Persian Restaurant,Categories_Peruvian Restaurant,Categories_Pet Café,Categories_Pie Shop,Categories_Pizza Place,Categories_Poke Place,Categories_Polish Restaurant,Categories_Portuguese Restaurant,Categories_Poutine Place,Categories_Ramen Restaurant,Categories_Restaurant,Categories_Russian Restaurant,Categories_Salad Place,Categories_Salvadoran Restaurant,Categories_Sandwich Place,Categories_Scandinavian Restaurant,Categories_Seafood Restaurant,Categories_Shanghai Restaurant,Categories_Shawarma Place,Categories_Snack Place,Categories_Soup Place,Categories_South American Restaurant,Categories_South Indian Restaurant,Categories_Southern / Soul Food Restaurant,Categories_Souvlaki Shop,Categories_Spanish Restaurant,Categories_Sri Lankan Restaurant,Categories_Steakhouse,Categories_Sushi Restaurant,Categories_Swiss Restaurant,Categories_Szechuan Restaurant,Categories_Taco Place,Categories_Taiwanese Restaurant,Categories_Tapas Restaurant,Categories_Tea Room,Categories_Tex-Mex Restaurant,Categories_Thai Restaurant,Categories_Theme Restaurant,Categories_Tibetan Restaurant,Categories_Turkish Restaurant,Categories_Udon Restaurant,Categories_Vegetarian / Vegan Restaurant,Categories_Vietnamese Restaurant,Categories_Wings Joint,Categories_Xinjiang Restaurant
0,H0M,0.0,0.0,0.0,0.0,0.

I'll also do a grouping calculating the sum, so later we can count the total number of venues in each cluster and city.

In [261]:
# group and calculate TOTAL of each category
places_sum = places_onehot.groupby('Place').sum().reset_index()

print(places_sum.shape)
places_sum.head()

(241, 140)


,Place,Categories_Afghan Restaurant,Categories_African Restaurant,Categories_American Restaurant,Categories_Arepa Restaurant,Categories_Argentinian Restaurant,Categories_Asian Restaurant,Categories_BBQ Joint,Categories_Bagel Shop,Categories_Bakery,Categories_Belgian Restaurant,Categories_Bistro,Categories_Brazilian Restaurant,Categories_Breakfast Spot,Categories_Bubble Tea Shop,Categories_Buffet,Categories_Burger Joint,Categories_Burmese Restaurant,Categories_Burrito Place,Categories_Cafeteria,Categories_Cajun / Creole Restaurant,Categories_Cambodian Restaurant,Categories_Cantonese Restaurant,Categories_Caribbean Restaurant,Categories_Chinese Breakfast Place,Categories_Chinese Restaurant,Categories_Colombian Restaurant,Categories_Comfort Food Restaurant,Categories_Creperie,Categories_Cuban Restaurant,Categories_Cupcake Shop,Categories_Deli / Bodega,Categories_Dessert Shop,Categories_Dim Sum Restaurant,Categories_Diner,Categories_Doner Restaurant,Categories_Dongbei Restaurant,Categories_Donut Shop,Categories_Dumpling Restaurant,Categories_Eastern European Restaurant,Categories_Egyptian Restaurant,Categories_Empanada Restaurant,Categories_English Restaurant,Categories_Ethiopian Restaurant,Categories_Falafel Restaurant,Categories_Fast Food Restaurant,Categories_Filipino Restaurant,Categories_Fish & Chips Shop,Categories_Fondue Restaurant,Categories_Food,Categories_Food Court,Categories_Food Stand,Categories_Food Truck,Categories_French Restaurant,Categories_Fried Chicken Joint,Categories_Frozen Yogurt Shop,Categories_Gastropub,Categories_German Restaurant,Categories_Gluten-free Restaurant,Categories_Greek Restaurant,Categories_Hakka Restaurant,Categories_Halal Restaurant,Categories_Hawaiian Restaurant,Categories_Hot Dog Joint,Categories_Hotpot Restaurant,Categories_Hungarian Restaurant,Categories_Ice Cream Shop,Categories_Indian Chinese Restaurant,Categories_Indian Restaurant,Categories_Indonesian Restaurant,Categories_Irish Pub,Categories_Israeli Restaurant,Categories_Italian Restaurant,Categories_Japanese Restaurant,Categories_Jewish Restaurant,Categories_Juice Bar,Categories_Kebab Restaurant,Categories_Korean Restaurant,Categories_Kosher Restaurant,Categories_Latin American Restaurant,Categories_Lebanese Restaurant,Categories_Mac & Cheese Joint,Categories_Malay Restaurant,Categories_Mediterranean Restaurant,Categories_Mexican Restaurant,Categories_Middle Eastern Restaurant,Categories_Modern European Restaurant,Categories_Mongolian Restaurant,Categories_Moroccan Restaurant,Categories_New American Restaurant,Categories_Noodle House,Categories_North Indian Restaurant,Categories_Pakistani Restaurant,Categories_Pastry Shop,Categories_Peking Duck Restaurant,Categories_Persian Restaurant,Categories_Peruvian Restaurant,Categories_Pet Café,Categories_Pie Shop,Categories_Pizza Place,Categories_Poke Place,Categories_Polish Restaurant,Categories_Portuguese Restaurant,Categories_Poutine Place,Categories_Ramen Restaurant,Categories_Restaurant,Categories_Russian Restaurant,Categories_Salad Place,Categories_Salvadoran Restaurant,Categories_Sandwich Place,Categories_Scandinavian Restaurant,Categories_Seafood Restaurant,Categories_Shanghai Restaurant,Categories_Shawarma Place,Categories_Snack Place,Categories_Soup Place,Categories_South American Restaurant,Categories_South Indian Restaurant,Categories_Southern / Soul Food Restaurant,Categories_Souvlaki Shop,Categories_Spanish Restaurant,Categories_Sri Lankan Restaurant,Categories_Steakhouse,Categories_Sushi Restaurant,Categories_Swiss Restaurant,Categories_Szechuan Restaurant,Categories_Taco Place,Categories_Taiwanese Restaurant,Categories_Tapas Restaurant,Categories_Tea Room,Categories_Tex-Mex Restaurant,Categories_Thai Restaurant,Categories_Theme Restaurant,Categories_Tibetan Restaurant,Categories_Turkish Restaurant,Categories_Udon Restaurant,Categories_Vegetarian / Vegan Restaurant,Categories_Vietnamese Restaurant,Categories_Wings Joint,Categories_Xinjiang Restaurant
0,H0M,0,0,0,0,0,0,0,0,0,

### k-means Cluster.

Run *k*-means to cluster postal codes based on proportion of food venues of each category

In [262]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans( n_clusters=kclusters, random_state=0).fit( places_mean.iloc[:,1:])

# preview cluster labels
print(len(kmeans.labels_))
print(kmeans.labels_[0:10])

241
[3 0 2 2 2 2 2 2 4 4]


### Top categories.

I'll use a function to sort the category columns in descending order, from the one hot table.

In [263]:
def top_categories_fx(row, num_categories):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_categories]

Now create the new dataframe and display the top venues for each postal code.

In [264]:
num_categories = 5

# create column names
number_suffixes = ['st', 'nd', 'rd']
columns = ['Place']
for n in np.arange(num_categories):
    if n<len(number_suffixes):
        columns.append('{}{} Most Frequent'.format(n+1, number_suffixes[n]))
    else:
        columns.append('{}th Most Frequent'.format(n+1))

# create a new dataframe
place_top_categories = pd.DataFrame(columns=columns)
place_top_categories['Place'] = places_mean['Place']

for n in np.arange( places_mean.shape[0]):
    place_top_categories.iloc[n, 1:] = top_categories_fx( places_mean.iloc[n, :], num_categories)

print(place_top_categories.shape)
place_top_categories.head()

(241, 6)


,Place,1st Most Frequent,2nd Most Frequent,3rd Most Frequent,4th Most Frequent,5th Most Frequent
0,H0M,Categories_Fast Food Restaurant,Categories_Italian Restaurant,Categories_Xinjiang Restaurant,Categories_Falafel Restaurant,Categories_Food Court
1,H1A,Categories_Pizza Place,Categories_Italian Restaurant,Categories_Asian Restaurant,Categories_Chinese Restaurant,Categories_Xinjiang Restaurant
2,H1B,Categories_Restaurant,Categories_Pizza Place,Categories_Sushi Restaurant,Categories_Donut Shop,Categories_Dessert Shop
3,H1C,Categories_Bakery,Categories_Food Truck,Categories_Food Court,Categories_Food,Categories_Fondue Restaurant
4,H1E,Categories_Italian Restaurant,Categories_Fast Food Restaurant,Categories_Pizza Place,Categories_Restaurant,Categories_Diner


### Create summary dataframes.

Combine the Postal Code location, cluster and top categories.

In [265]:
# join to place coords
place_clusters = pd.merge(places_coords, place_top_categories, left_on='Place', right_on='Place')
# add cluster labels
place_clusters.insert(4, 'Cluster', kmeans.labels_)
place_clusters.sort_values(by=['Cluster'], inplace=True) # sort by cluster

print(place_clusters.shape)
place_clusters.head()

(241, 10)


,City,Place,Place Latitude,Place Longitude,Cluster,1st Most Frequent,2nd Most Frequent,3rd Most Frequent,4th Most Frequent,5th Most Frequent
120,"Montreal, Quebec",H1Y,45.5486,-73.5788,0,Categories_Fast Food Restaurant,Categories_Asian Restaurant,Categories_Breakfast Spot,Categories_Italian Restaurant,Categories_Pizza Place
98,"Toronto, Ontario",M9R,43.6898,-79.5582,0,Categories_Pizza Place,Categories_Asian Restaurant,Categories_Sandwich Place,Categories_Chinese Restaurant,Categories_American Restaurant
101,"Montreal, Quebec",H0M,45.6986,-73.5025,0,Categories_Fast Food Restaurant,Categories_Italian Restaurant,Categories_Xinjiang Restaurant,Categories_Falafel Restaurant,Categories_Food Court
105,"Montreal, Quebec",H1E,45.6342,-73.5842,0,Categories_Italian Restaurant,Categories_Fast Food Restaurant,Categories_Pizza Place,Categories_Restaurant,Categories_Diner
106,"Montreal, Quebec",H1G,45.6109,-73.6211,0,Categories_Fast Food Restaurant,Categories_Asian Restaurant,Categories_Restaurant,Categories_Sandwich Place,Categories_Breakfast Spot


#### Group by cluster.

One-hot to count the number of places from each city in each cluster.

In [266]:
# one hot encoding cities
cities_onehot = pd.get_dummies(place_clusters['City'])

# add cluster back to dataframe
cities_onehot = pd.merge(place_clusters['Cluster'], cities_onehot, left_index=True, right_index=True) #merge on index
cities_onehot = cities_onehot.groupby('Cluster').sum().reset_index()

print(cities_onehot.shape)
cities_onehot.head()

(5, 4)


,Cluster,"Calgary, Alberta","Montreal, Quebec","Toronto, Ontario"
0,0,7,21,4
1,1,2,11,1
2,2,7,45,61
3,3,2,2,4
4,4,7,36,31


Take the one hot category sums grouped by postal code and group by cluster and calculate the mean across postal code venue counts.

In [267]:
cluster_sum = place_clusters[['Cluster','Place']]

# join to category sums
cluster_sum = pd.merge(cluster_sum, places_sum, left_on='Place', right_on='Place')
cluster_sum.drop(['Place'], axis=1)

cluster_mean = cluster_sum.groupby('Cluster').mean().reset_index()

print(cluster_mean.shape)
cluster_mean.head()

(5, 140)


,Cluster,Categories_Afghan Restaurant,Categories_African Restaurant,Categories_American Restaurant,Categories_Arepa Restaurant,Categories_Argentinian Restaurant,Categories_Asian Restaurant,Categories_BBQ Joint,Categories_Bagel Shop,Categories_Bakery,Categories_Belgian Restaurant,Categories_Bistro,Categories_Brazilian Restaurant,Categories_Breakfast Spot,Categories_Bubble Tea Shop,Categories_Buffet,Categories_Burger Joint,Categories_Burmese Restaurant,Categories_Burrito Place,Categories_Cafeteria,Categories_Cajun / Creole Restaurant,Categories_Cambodian Restaurant,Categories_Cantonese Restaurant,Categories_Caribbean Restaurant,Categories_Chinese Breakfast Place,Categories_Chinese Restaurant,Categories_Colombian Restaurant,Categories_Comfort Food Restaurant,Categories_Creperie,Categories_Cuban Restaurant,Categories_Cupcake Shop,Categories_Deli / Bodega,Categories_Dessert Shop,Categories_Dim Sum Restaurant,Categories_Diner,Categories_Doner Restaurant,Categories_Dongbei Restaurant,Categories_Donut Shop,Categories_Dumpling Restaurant,Categories_Eastern European Restaurant,Categories_Egyptian Restaurant,Categories_Empanada Restaurant,Categories_English Restaurant,Categories_Ethiopian Restaurant,Categories_Falafel Restaurant,Categories_Fast Food Restaurant,Categories_Filipino Restaurant,Categories_Fish & Chips Shop,Categories_Fondue Restaurant,Categories_Food,Categories_Food Court,Categories_Food Stand,Categories_Food Truck,Categories_French Restaurant,Categories_Fried Chicken Joint,Categories_Frozen Yogurt Shop,Categories_Gastropub,Categories_German Restaurant,Categories_Gluten-free Restaurant,Categories_Greek Restaurant,Categories_Hakka Restaurant,Categories_Halal Restaurant,Categories_Hawaiian Restaurant,Categories_Hot Dog Joint,Categories_Hotpot Restaurant,Categories_Hungarian Restaurant,Categories_Ice Cream Shop,Categories_Indian Chinese Restaurant,Categories_Indian Restaurant,Categories_Indonesian Restaurant,Categories_Irish Pub,Categories_Israeli Restaurant,Categories_Italian Restaurant,Categories_Japanese Restaurant,Categories_Jewish Restaurant,Categories_Juice Bar,Categories_Kebab Restaurant,Categories_Korean Restaurant,Categories_Kosher Restaurant,Categories_Latin American Restaurant,Categories_Lebanese Restaurant,Categories_Mac & Cheese Joint,Categories_Malay Restaurant,Categories_Mediterranean Restaurant,Categories_Mexican Restaurant,Categories_Middle Eastern Restaurant,Categories_Modern European Restaurant,Categories_Mongolian Restaurant,Categories_Moroccan Restaurant,Categories_New American Restaurant,Categories_Noodle House,Categories_North Indian Restaurant,Categories_Pakistani Restaurant,Categories_Pastry Shop,Categories_Peking Duck Restaurant,Categories_Persian Restaurant,Categories_Peruvian Restaurant,Categories_Pet Café,Categories_Pie Shop,Categories_Pizza Place,Categories_Poke Place,Categories_Polish Restaurant,Categories_Portuguese Restaurant,Categories_Poutine Place,Categories_Ramen Restaurant,Categories_Restaurant,Categories_Russian Restaurant,Categories_Salad Place,Categories_Salvadoran Restaurant,Categories_Sandwich Place,Categories_Scandinavian Restaurant,Categories_Seafood Restaurant,Categories_Shanghai Restaurant,Categories_Shawarma Place,Categories_Snack Place,Categories_Soup Place,Categories_South American Restaurant,Categories_South Indian Restaurant,Categories_Southern / Soul Food Restaurant,Categories_Souvlaki Shop,Categories_Spanish Restaurant,Categories_Sri Lankan Restaurant,Categories_Steakhouse,Categories_Sushi Restaurant,Categories_Swiss Restaurant,Categories_Szechuan Restaurant,Categories_Taco Place,Categories_Taiwanese Restaurant,Categories_Tapas Restaurant,Categories_Tea Room,Categories_Tex-Mex Restaurant,Categories_Thai Restaurant,Categories_Theme Restaurant,Categories_Tibetan Restaurant,Categories_Turkish Restaurant,Categories_Udon Restaurant,Categories_Vegetarian / Vegan Restaurant,Categories_Vietnamese Restaurant,Categories_Wings Joint,Categories_Xinjiang Restaurant
0,0,0.000000,0.031250,

Create another summary dataframe combiing the city counts with the most frequent categories, but by cluster.

In [268]:
num_categories = 5

# create column names
number_suffixes = ['st', 'nd', 'rd']
columns = ['Cluster']
for n in np.arange(num_categories):
    if n<len(number_suffixes):
        columns.append('{}{} Most Frequent'.format(n+1, number_suffixes[n]))
    else:
        columns.append('{}th Most Frequent'.format(n+1))

# create a new dataframe
cluster_top_categories = pd.DataFrame(columns=columns)
cluster_top_categories['Cluster'] = cluster_mean['Cluster']

for n in np.arange( cluster_mean.shape[0]):
    cluster_top_categories.iloc[n, 1:] = top_categories_fx( cluster_mean.iloc[n, :], num_categories)

cluster_summary = pd.merge(cities_onehot, cluster_top_categories, left_on='Cluster', right_on='Cluster')
    
print(cluster_summary.shape)
cluster_summary.head()

(5, 9)


,Cluster,"Calgary, Alberta","Montreal, Quebec","Toronto, Ontario",1st Most Frequent,2nd Most Frequent,3rd Most Frequent,4th Most Frequent,5th Most Frequent
0,0,7,21,4,Categories_Asian Restaurant,Categories_Pizza Place,Categories_Fast Food Restaurant,Categories_Restaurant,Categories_Dessert Shop
1,1,2,11,1,Categories_Asian Restaurant,Categories_Fast Food Restaurant,Categories_Pizza Place,Categories_Dessert Shop,Categories_Restaurant
2,2,7,45,61,Categories_Asian Restaurant,Categories_Pizza Place,Categories_Fast Food Restaurant,Categories_Restaurant,Categories_Bakery
3,3,2,2,4,Categories_Asian Restaurant,Categories_Fast Food Restaurant,Categories_Restaurant,Categories_Japanese Restaurant,Categories_Italian Restaurant
4,4,7,36,31,Categories_Asian Restaurant,Categories_Fast Food Restaurant,Categories_Restaurant,Categories_Pizza Place,Categories_Japanese Restaurant


We can see that Asian restaurants are the most frequent across all clusters, followed by pizza places, fast food and the overall restaurant category. However once we get into the 4th and 5th most frequent categories differences in clusters emerge.

#### Group by city.

As above, but by city.

In [269]:
# one hot encoding clusters
clusters_onehot = pd.get_dummies(place_clusters['Cluster'], prefix="Cluster", prefix_sep="")

# add city back to onehot of clusters and group by city
clusters_onehot = pd.merge(place_clusters['City'], clusters_onehot, left_index=True, right_index=True) #merge on index
clusters_onehot = clusters_onehot.groupby('City').sum().reset_index()


# join city to category 
city_sum = place_clusters[['City','Place']]
city_sum = pd.merge(city_sum, places_sum, left_on='Place', right_on='Place')
city_sum.drop(['Place'], axis=1)

# group by city, calculating grand total venues of each category per city
city_mean = city_sum.groupby('City').mean().reset_index()

print(city_mean.shape)
city_mean.head()

(3, 140)


,City,Categories_Afghan Restaurant,Categories_African Restaurant,Categories_American Restaurant,Categories_Arepa Restaurant,Categories_Argentinian Restaurant,Categories_Asian Restaurant,Categories_BBQ Joint,Categories_Bagel Shop,Categories_Bakery,Categories_Belgian Restaurant,Categories_Bistro,Categories_Brazilian Restaurant,Categories_Breakfast Spot,Categories_Bubble Tea Shop,Categories_Buffet,Categories_Burger Joint,Categories_Burmese Restaurant,Categories_Burrito Place,Categories_Cafeteria,Categories_Cajun / Creole Restaurant,Categories_Cambodian Restaurant,Categories_Cantonese Restaurant,Categories_Caribbean Restaurant,Categories_Chinese Breakfast Place,Categories_Chinese Restaurant,Categories_Colombian Restaurant,Categories_Comfort Food Restaurant,Categories_Creperie,Categories_Cuban Restaurant,Categories_Cupcake Shop,Categories_Deli / Bodega,Categories_Dessert Shop,Categories_Dim Sum Restaurant,Categories_Diner,Categories_Doner Restaurant,Categories_Dongbei Restaurant,Categories_Donut Shop,Categories_Dumpling Restaurant,Categories_Eastern European Restaurant,Categories_Egyptian Restaurant,Categories_Empanada Restaurant,Categories_English Restaurant,Categories_Ethiopian Restaurant,Categories_Falafel Restaurant,Categories_Fast Food Restaurant,Categories_Filipino Restaurant,Categories_Fish & Chips Shop,Categories_Fondue Restaurant,Categories_Food,Categories_Food Court,Categories_Food Stand,Categories_Food Truck,Categories_French Restaurant,Categories_Fried Chicken Joint,Categories_Frozen Yogurt Shop,Categories_Gastropub,Categories_German Restaurant,Categories_Gluten-free Restaurant,Categories_Greek Restaurant,Categories_Hakka Restaurant,Categories_Halal Restaurant,Categories_Hawaiian Restaurant,Categories_Hot Dog Joint,Categories_Hotpot Restaurant,Categories_Hungarian Restaurant,Categories_Ice Cream Shop,Categories_Indian Chinese Restaurant,Categories_Indian Restaurant,Categories_Indonesian Restaurant,Categories_Irish Pub,Categories_Israeli Restaurant,Categories_Italian Restaurant,Categories_Japanese Restaurant,Categories_Jewish Restaurant,Categories_Juice Bar,Categories_Kebab Restaurant,Categories_Korean Restaurant,Categories_Kosher Restaurant,Categories_Latin American Restaurant,Categories_Lebanese Restaurant,Categories_Mac & Cheese Joint,Categories_Malay Restaurant,Categories_Mediterranean Restaurant,Categories_Mexican Restaurant,Categories_Middle Eastern Restaurant,Categories_Modern European Restaurant,Categories_Mongolian Restaurant,Categories_Moroccan Restaurant,Categories_New American Restaurant,Categories_Noodle House,Categories_North Indian Restaurant,Categories_Pakistani Restaurant,Categories_Pastry Shop,Categories_Peking Duck Restaurant,Categories_Persian Restaurant,Categories_Peruvian Restaurant,Categories_Pet Café,Categories_Pie Shop,Categories_Pizza Place,Categories_Poke Place,Categories_Polish Restaurant,Categories_Portuguese Restaurant,Categories_Poutine Place,Categories_Ramen Restaurant,Categories_Restaurant,Categories_Russian Restaurant,Categories_Salad Place,Categories_Salvadoran Restaurant,Categories_Sandwich Place,Categories_Scandinavian Restaurant,Categories_Seafood Restaurant,Categories_Shanghai Restaurant,Categories_Shawarma Place,Categories_Snack Place,Categories_Soup Place,Categories_South American Restaurant,Categories_South Indian Restaurant,Categories_Southern / Soul Food Restaurant,Categories_Souvlaki Shop,Categories_Spanish Restaurant,Categories_Sri Lankan Restaurant,Categories_Steakhouse,Categories_Sushi Restaurant,Categories_Swiss Restaurant,Categories_Szechuan Restaurant,Categories_Taco Place,Categories_Taiwanese Restaurant,Categories_Tapas Restaurant,Categories_Tea Room,Categories_Tex-Mex Restaurant,Categories_Thai Restaurant,Categories_Theme Restaurant,Categories_Tibetan Restaurant,Categories_Turkish Restaurant,Categories_Udon Restaurant,Categories_Vegetarian / Vegan Restaurant,Categories_Vietnamese Restaurant,Categories_Wings Joint,Categories_Xinjiang Restaurant
0,"Calgary, Alberta",0.00

In [270]:
# get top categories by city
num_categories = 5

# create column names
number_suffixes = ['st', 'nd', 'rd']
columns = ['City']
for n in np.arange(num_categories):
    if n<len(number_suffixes):
        columns.append('{}{} Most Frequent'.format(n+1, number_suffixes[n]))
    else:
        columns.append('{}th Most Frequent'.format(n+1))

# create a new dataframe of top categories
city_top_categories = pd.DataFrame(columns=columns)
city_top_categories['City'] = city_mean['City']

for n in np.arange( city_mean.shape[0]):
    city_top_categories.iloc[n, 1:] = top_categories_fx( city_mean.iloc[n, :], num_categories)

# merge clusters and top categories
city_summary = pd.merge(clusters_onehot, city_top_categories, left_on='City', right_on='City')
    
print(city_summary.shape)
city_summary.head()

(3, 11)


,City,Cluster0,Cluster1,Cluster2,Cluster3,Cluster4,1st Most Frequent,2nd Most Frequent,3rd Most Frequent,4th Most Frequent,5th Most Frequent
0,"Calgary, Alberta",7,2,7,2,7,Categories_Asian Restaurant,Categories_Pizza Place,Categories_Fast Food Restaurant,Categories_Sandwich Place,Categories_Vietnamese Restaurant
1,"Montreal, Quebec",21,11,45,2,36,Categories_Asian Restaurant,Categories_Fast Food Restaurant,Categories_Restaurant,Categories_Pizza Place,Categories_Dessert Shop
2,"Toronto, Ontario",4,1,61,4,31,Categories_Asian Restaurant,Categories_Pizza Place,Categories_Fast Food Restaurant,Categories_Restaurant,Categories_Japanese Restaurant


## Mapping

#### Use geopy library to get the latitude and longitude values of the city.

In [271]:
geolocator = Nominatim(user_agent="explorer") #define a user_agent.

latitude=[]
longitude=[]
for city in city_codes['City']: # loop through cities list
    location = geolocator.geocode(city) # geocode query
    latitude.append(location.latitude)
    longitude.append(location.longitude)

# add to cities dataframe
city_codes['Latitude']=latitude
city_codes['Longitude']=longitude
city_codes

,Letter,City,Latitude,Longitude
0,M,"Toronto, Ontario",43.653963,-79.387207
1,H,"Montreal, Quebec",45.497216,-73.610364
2,T,"Calgary, Alberta",51.025327,-114.049868


#### Create a map of postal codes

In [272]:
for city, c_lat, c_lng in zip(city_codes['City'], city_codes['Latitude'], city_codes['Longitude']):
    
    marker_data = zip(
        places_coords[places_coords['City']==city]['Place'],
        places_coords[places_coords['City']==city]['Place Latitude'],
        places_coords[places_coords['City']==city]['Place Longitude'])
    
    # create map using latitude and longitude values
    map_pc = folium.Map(location=[c_lat, c_lng], zoom_start=11)

    # add markers to map
    for postcode, p_lat, p_lng in marker_data:

        label = postcode
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [p_lat, p_lng],
            radius=5,
            popup=label,
            fill=True,
            fill_opacity=0.7,
            parse_html=False).add_to(map_pc)  

    display(map_pc)

#### Create a map of venues

In [273]:
for city, c_lat, c_lng in zip(city_codes['City'], city_codes['Latitude'], city_codes['Longitude']):
    
    marker_data = venues_kept[venues_kept['City']==city][['Latitude','Longitude']]

    # create map
    map_venue = folium.Map(location=[c_lat, c_lng], zoom_start=11)

    # using FastMarkerCluster
    map_venue.add_child(FastMarkerCluster(marker_data.values.tolist()))

    display(map_venue)

## Visualize Clusters

In [274]:
for city, c_lat, c_lng in zip(city_codes['City'], city_codes['Latitude'], city_codes['Longitude']):
    
    marker_data = zip(
        place_clusters[place_clusters['City']==city]['Place'],
        place_clusters[place_clusters['City']==city]['Cluster'],
        place_clusters[place_clusters['City']==city]['1st Most Frequent'],
        place_clusters[place_clusters['City']==city]['Place Latitude'],
        place_clusters[place_clusters['City']==city]['Place Longitude'])

    # create map
    map_clusters = folium.Map(location=[c_lat, c_lng], zoom_start=11)

    # set color scheme for the clusters
    k_array = np.arange(kclusters)
    colors_array = cm.Set1(np.linspace(0, 1, 9)) # Set1 color map has 9 colors
    color_map = [colors.rgb2hex(i) for i in colors_array]
    # markers_colors = []

    # add markers to the map

    for place, cluster, category, p_lat, p_lng in marker_data:

        label = folium.Popup( '{}\nCluster {}\n{}.'.format(place, cluster, category), parse_html=True)

        folium.CircleMarker(
            [p_lat, p_lng],
            radius=5,
            popup=label,
            color=color_map[cluster%9],
            fill=True,
            fill_color=color_map[cluster%9],
            fill_opacity=0.7).add_to(map_clusters)

    # make legend in html/css from color map
    legend_str=''
    for item, color in zip(k_array, color_map[:kclusters]):
        legend_str+='<tr><td>Cluster {}</td><td><i class="fa fa-circle" style="color:{}"></i></td>'.format(item, color)
    legend_html = '''
        <div style="
            z-index:9999;
            position: fixed; 
            bottom: 40px; 
            right: 40px; 
            padding: 6px;
            border:1px solid grey; 
            font-size:14px;">
        <b>Legend</b>
        <table>
            {}
        </table>
        </div> '''.format(legend_str)
    map_clusters.get_root().html.add_child(folium.Element(legend_html))

    display(map_clusters)

Thank you for viewing my work.